# GabeDA Features (Daily-Hour Aggregation)

This notebook creates hourly metrics aggregated by date and hour.
It processes transaction data to generate hourly snapshots with cross-model comparisons.

**Input:** Preprocessed transactions from 01_transactions notebook  
**Output:** Hourly aggregated metrics (1 row per day + hour combination)  
**Group By:** `dt_date`, `hour`

## 1. Setup: Imports, Context Loading, Logging

## 0. Project Root Setup (Auto-generated)

In [1]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/development or notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

Working directory: c:\Projects\play\khujta_ai_business
Project root: c:\Projects\play\khujta_ai_business


In [2]:
import pandas as pd
import numpy as np

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.core.persistence import load_context_state, get_latest_state, save_context_state
from src.core.constants import *
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.export.excel import ExcelExporter

# Load latest context state
client_name = 'test_client'
latest_state = get_latest_state(client_name, base_dir='data/context_states')

if latest_state:
    ctx, base_cfg = load_context_state(latest_state)
    print(f"✓ Loaded latest state: {latest_state}")
else:
    raise FileNotFoundError(f"No context state found for client '{client_name}'")

# Setup logging
setup_logging(log_level=base_cfg.get('log_level', 'INFO'), 
              config={'client': base_cfg.get('client', 'unknown_client')})
logger = get_logger(__name__)

print(f"\n✓ Context loaded successfully!")
print(f"  - Original run_id: {ctx.original_run_id}")
print(f"  - New run_id: {ctx.run_id}")
print(f"  - Available datasets: {len(ctx.list_datasets())} datasets")

✓ Loaded latest state: data\context_states\test_client_20251022_150907
📝 Run instance ID: test_client_20251022_150949 - Logging [INFO] to: logs\test_client_20251022_150949.log

✓ Context loaded successfully!
  - Original run_id: test_client_20251022_150937
  - New run_id: test_client_20251022_150949
  - Available datasets: 5 datasets


## 2. Load Input Data

In [3]:
# Get input dataset
input_df = ctx.get_dataset('transactions_filters')

print(f"✓ Input dataset loaded")
print(f"  - Shape: {input_df.shape}")
print(f"  - Date range: {input_df['dt_date'].min()} to {input_df['dt_date'].max()}")
print(f"  - Unique dates: {input_df['dt_date'].nunique()}")
print(f"  - Unique hours: {input_df['hour'].nunique()}")
print(f"\nFirst few rows:")
input_df.head()

✓ Input dataset loaded
  - Shape: (609, 59)
  - Date range: 20251001 to 20251030
  - Unique dates: 30
  - Unique hours: 23

First few rows:


,in_dt,in_product_id,in_quantity,in_price_total,in_trans_type,in_customer_id,in_description,in_category,in_unit_type,in_stock,...,cost_unit,cost_total,price_unit,price_total,margin_unit,margin_unit_pct,margin_unit_valid,margin_total,margin_total_pct,margin_total_valid
0,2025-10-01 01:02:00,prod8,2.0,52964.0,return,client13,product 8,category B,pack,61.0,...,18792.0,37585.0,26482.0,52964.0,7690.0,29.04,True,15379.0,29.04,True
1,2025-10-01 06:24:00,prod4,6.0,177195.0,sale,client6,product 4,category B,unit,30.0,...,21526.0,129155.0,29533.0,177195.0,8007.0,27.11,True,48040.0,27.11,True
2,2025-10-01 08:38:00,prod7,2.0,70492.0,return,client12,product 7,category A,unit,78.0,...,25754.0,51509.0,35246.0,70492.0,9492.0,26.93,True,18983.0,26.93,True
3,2025-10-01 09:59:00,prod2,4.0,86751.0,sale,client3,product 2,category A,unit,80.0,...,12947.0,51786.0,21688.0,86751.0,8741.0,40.30,True,34965.0,40.31,True
4,2025-10-01 10:07:00,prod3,3.0,76465.0,sale,client12,product 3,category B,unit,47.0,...,16943.0,5083.0,25488.0,76465.0,8545.0,33.53,True,71382.0,93.35,True


## 3. Define Features

Hourly metrics aggregated by date and hour:  
- **Filters:** Transaction categorization (ticket size, volume, time segment)
- **Attributes:** Aggregated metrics (counts, sums, averages, medians)
- **Cross-model comparisons:** Hour vs day comparisons using external data

In [4]:
# ===== FILTERS (Row-Level Calculations) =====

def ticket_size_cat(in_price_total):
    """
    Categorize transaction by ticket size.
    Formula: Low (<10k), Medium (10k-50k), High (>50k)
    """
    if pd.isna(in_price_total):
        return 'Unknown'
    if in_price_total < 10000:
        return 'Low'
    elif in_price_total < 50000:
        return 'Medium'
    else:
        return 'High'

def volume_cat(in_quantity):
    """
    Categorize transaction by quantity.
    Formula: Unit (<3), Multiple (3-5), Massive (>5)
    """
    if in_quantity < 3:
        return 'Unit'
    elif in_quantity < 6:
        return 'Multiple'
    else:
        return 'Massive'

def day_segment(timestamp):
    """
    Classify time of day based on hour.
    Formula: Morning (5-12), Afternoon (12-17), Evening (17-21), Night (21-5)
    """
    hour = pd.Timestamp(timestamp).hour
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

# ===== ATTRIBUTES (Aggregated Calculations) =====

def weekday_name(dt_weekday_name):
    """
    Get weekday name (takes first value since all rows in group have same date).
    Formula: FIRST(dt_weekday_name)
    """
    return dt_weekday_name[FIRST_VALUE]

# Identifiers
def customer_id_count(customer_id):
    """Count unique customer IDs."""
    return len(np.unique(customer_id))

def trans_id_count(trans_id):
    """Count unique transaction IDs."""
    return len(np.unique(trans_id))

def product_id_count(product_id):
    """Count unique product IDs."""
    return len(np.unique(product_id))

# Quantity Metrics
def quantity_sum(quantity):
    """Total quantity sold."""
    return np.sum(quantity)

def quantity_mean(quantity):
    """Average quantity per transaction."""
    return round(np.mean(quantity), 2)

def quantity_median(quantity):
    """Median quantity per transaction."""
    return round(np.median(quantity), 2)

# Price Metrics
def price_total_sum(price_total):
    """Total revenue."""
    return np.sum(price_total)

def price_total_mean(price_total):
    """Average transaction price."""
    return round(np.mean(price_total), 2)

def price_total_median(price_total):
    """Median transaction price."""
    return round(np.median(price_total), 2)

# Cost Metrics
def cost_total_sum(cost_total):
    """Total cost."""
    return np.sum(cost_total)

def cost_total_mean(cost_total):
    """Average cost per transaction."""
    return round(np.mean(cost_total), 2)

def cost_total_median(cost_total):
    """Median cost per transaction."""
    return round(np.median(cost_total), 2)

def transaction_mean(price_total_sum, trans_id_count):
    """
    Average spend per transaction.
    Formula: price_total_sum / trans_id_count
    """
    if trans_id_count == 0:
        return DEFAULT_FLOAT
    return round(np.mean(price_total_sum / trans_id_count), 2)

# Margin Metrics (Total)
def margin_total_sum(margin_total, margin_total_pct, margin_total_valid):
    """Sum of valid margin totals (excluding DEFAULT_FLOAT cases)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total[flag]
    return np.sum(select) if len(select) > 0 else DEFAULT_FLOAT

def margin_total_mean(margin_total, margin_total_pct, margin_total_valid):
    """Average of valid margin totals (excluding DEFAULT_FLOAT cases)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total[flag]
    return round(np.mean(select), 2) if len(select) > 0 else DEFAULT_FLOAT

def margin_total_median(margin_total, margin_total_pct, margin_total_valid):
    """Median of valid margin totals (excluding DEFAULT_FLOAT cases)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total[flag]
    return round(np.median(select), 2) if len(select) > 0 else DEFAULT_FLOAT

# Margin Metrics (Percentage)
def margin_total_pct_min(margin_total_pct, margin_total_valid):
    """Minimum valid margin percentage (excluding DEFAULT_FLOAT = -16.0)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total_pct[flag]
    return round(np.min(select), 2) if len(select) > 0 else DEFAULT_FLOAT

def margin_total_pct_mean(margin_total_pct, margin_total_valid):
    """Average valid margin percentage (excluding DEFAULT_FLOAT = -16.0)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total_pct[flag]
    return round(np.mean(select), 2) if len(select) > 0 else DEFAULT_FLOAT

def margin_total_pct_median(margin_total_pct, margin_total_valid):
    """Median valid margin percentage (excluding DEFAULT_FLOAT = -16.0)."""
    flag = (margin_total_pct != DEFAULT_FLOAT) & (margin_total_valid == True)
    select = margin_total_pct[flag]
    return round(np.median(select), 2) if len(select) > 0 else DEFAULT_FLOAT

# ===== CROSS-MODEL COMPARISONS (Hour vs Day) =====
# These features use external_data to merge daily_attrs and compare hourly metrics against daily baselines

def customer_id_vs_daily_count_pct(customer_id_count: int, daily_attrs_customer_id_count: int) -> float:
    """
    Calculate percentage deviation of hourly customer count from daily customer count.
    Formula: (hourly - daily) / daily * 100
    """
    if daily_attrs_customer_id_count == 0:
        return DEFAULT_FLOAT
    return round((customer_id_count - daily_attrs_customer_id_count) / daily_attrs_customer_id_count * 100, 2)

def product_id_vs_daily_count_pct(product_id_count: int, daily_attrs_product_id_count: int) -> float:
    """
    Calculate percentage deviation of hourly product count from daily product count.
    Formula: (hourly - daily) / daily * 100
    """
    if daily_attrs_product_id_count == 0:
        return DEFAULT_FLOAT
    return round((product_id_count - daily_attrs_product_id_count) / daily_attrs_product_id_count * 100, 2)

def trans_id_vs_daily_count_pct(trans_id_count: int, daily_attrs_trans_id_count: int) -> float:
    """
    Calculate percentage deviation of hourly transaction count from daily transaction count.
    Formula: (hourly - daily) / daily * 100
    """
    if daily_attrs_trans_id_count == 0:
        return DEFAULT_FLOAT
    return round((trans_id_count - daily_attrs_trans_id_count) / daily_attrs_trans_id_count * 100, 2)

print("✓ Feature functions defined: 26 features (3 filters + 23 attributes)")

✓ Feature functions defined: 26 features (3 filters + 23 attributes)


## 4. Configure Model

In [5]:
# Collect features into dictionary
features = {
    # Filters
    'ticket_size_cat': ticket_size_cat,
    'volume_cat': volume_cat,
    'day_segment': day_segment,
    # Attributes
    'weekday_name': weekday_name,
    'customer_id_count': customer_id_count,
    'trans_id_count': trans_id_count,
    'product_id_count': product_id_count,
    'quantity_sum': quantity_sum,
    'quantity_mean': quantity_mean,
    'quantity_median': quantity_median,
    'price_total_sum': price_total_sum,
    'price_total_mean': price_total_mean,
    'price_total_median': price_total_median,
    'cost_total_sum': cost_total_sum,
    'cost_total_mean': cost_total_mean,
    'cost_total_median': cost_total_median,
    'transaction_mean': transaction_mean,
    'margin_total_sum': margin_total_sum,
    'margin_total_mean': margin_total_mean,
    'margin_total_median': margin_total_median,
    'margin_total_pct_min': margin_total_pct_min,
    'margin_total_pct_mean': margin_total_pct_mean,
    'margin_total_pct_median': margin_total_pct_median,
    'customer_id_vs_daily_count_pct': customer_id_vs_daily_count_pct,
    'product_id_vs_daily_count_pct': product_id_vs_daily_count_pct,
    'trans_id_vs_daily_count_pct': trans_id_vs_daily_count_pct,
}

# Model configuration
cfg_model = {
    'model_name': 'daily_hour',
    'input_dataset_name': 'transactions_filters',
    'group_by': ['dt_date', 'hour'],  # Hourly aggregation grouped by date and hour
    'row_id': 'in_trans_id',
    'output_cols': list(features.keys()),
    'features': features,
    # External data configuration: Merge daily_attrs for cross-aggregation comparisons
    'external_data': {
        'daily_attrs': {
            'source': 'daily_attrs',           # Dataset name in context
            'join_on': ['dt_date'],            # Join on date (broadcast daily to hourly)
            'columns': None                    # None = bring ALL columns
        }
    }
}

print(f"✓ Model configured: '{cfg_model['model_name']}'")
print(f"  - Group by: {cfg_model['group_by']}")
print(f"  - Output features: {len(cfg_model['output_cols'])}")
print(f"  - External data sources: {list(cfg_model.get('external_data', {}).keys())}")

✓ Model configured: 'daily_hour'
  - Group by: ['dt_date', 'hour']
  - Output features: 26
  - External data sources: ['daily_attrs']


## 5. Prepare Features (Store, Resolve Dependencies, Save Config)

In [6]:
# Initialize feature store and store features
feature_store = FeatureStore()
feature_store.store_features(features, model_name=cfg_model['model_name'], auto_save=True)

# Resolve dependencies
resolver = DependencyResolver(feature_store)
in_cols, exec_seq, ext_cols = resolver.resolve_dependencies(
    output_cols=cfg_model['output_cols'],
    available_cols=input_df.columns.tolist(),
    group_by=cfg_model.get('group_by'),
    model=cfg_model['model_name'],
    external_data=cfg_model.get('external_data')
)

# Update model config with resolved dependencies
cfg_model['in_cols'] = in_cols
cfg_model['exec_seq'] = exec_seq
cfg_model['ext_cols'] = ext_cols

# Save master configuration
feature_store.save_master_config(
    model_name=cfg_model['model_name'],
    model_config=cfg_model
)

print("✓ Features prepared and dependencies resolved")
print(f"  - Input columns needed: {len(in_cols)}")
print(f"  - Execution sequence: {len(exec_seq)} features")
if ext_cols and ext_cols.get('dict'):
    print(f"  - External columns from daily_attrs: {ext_cols['dict']['daily_attrs']}")
print(f"  - Master config saved: feature_store/{cfg_model['model_name']}/master_cfg.json")

✓ Features prepared and dependencies resolved
  - Input columns needed: 16
  - Execution sequence: 26 features
  - External columns from daily_attrs: ['customer_id_count', 'product_id_count', 'trans_id_count']
  - Master config saved: feature_store/daily_hour/master_cfg.json


## 6. Execute Model (Initialize Components + Execute + Store Results)

In [7]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor, context=ctx)

# Execute model
output = executor.execute_model(
    cfg_model=cfg_model,
    input_dataset_name=cfg_model['input_dataset_name']
)

# Store results in context
ctx.set_model_output(cfg_model['model_name'], output, cfg_model)

print("✓ Model executed successfully!")
print(f"  - Filters: {output['filters'].shape if output['filters'] is not None else 'None'}")
print(f"  - Attributes: {output['attrs'].shape if output['attrs'] is not None else 'None'}")
print(f"  - Hourly snapshots created: {output['attrs'].shape[0] if output['attrs'] is not None else 0}")

✓ Model executed successfully!
  - Filters: (609, 85)
  - Attributes: (301, 25)
  - Hourly snapshots created: 301


## 7. View Results

In [8]:
# View hourly attributes (aggregated metrics)
attrs = ctx.get_model_attrs(cfg_model['model_name'])
print(f"Hourly Aggregated Metrics (n={len(attrs)}):")
attrs.head(10)

Hourly Aggregated Metrics (n=301):


,dt_date,hour,weekday_name,customer_id_count,trans_id_count,product_id_count,quantity_sum,quantity_mean,quantity_median,price_total_sum,...,transaction_mean,margin_total_sum,margin_total_mean,margin_total_median,margin_total_pct_min,margin_total_pct_mean,margin_total_pct_median,customer_id_vs_daily_count_pct,product_id_vs_daily_count_pct,trans_id_vs_daily_count_pct
0,20251001,1,Wednesday,1,1,1,2,2.00,2.0,52964.0,...,52964.00,15379.0,15379.00,15379.0,29.04,29.04,29.04,-92.86,-88.89,-95.45
1,20251001,6,Wednesday,1,1,1,6,6.00,6.0,177195.0,...,177195.00,48040.0,48040.00,48040.0,27.11,27.11,27.11,-92.86,-88.89,-95.45
2,20251001,8,Wednesday,1,1,1,2,2.00,2.0,70492.0,...,70492.00,18983.0,18983.00,18983.0,26.93,26.93,26.93,-92.86,-88.89,-95.45
3,20251001,9,Wednesday,1,1,1,4,4.00,4.0,86751.0,...,86751.00,34965.0,34965.00,34965.0,40.31,40.31,40.31,-92.86,-88.89,-95.45
4,20251001,10,Wednesday,1,1,1,3,3.00,3.0,76465.0,...,76465.00,71382.0,71382.00,71382.0,93.35,93.35,93.35,-92.86,-88.89,-95.45
5,20251001,11,Wednesday,3,3,3,11,3.67,4.0,160588.0,...,53529.33,60649.0,30324.50,30324.5,35.07,38.88,38.88,-78.57,-66.67,-86.36
6,20251001,14,Wednesday,3,4,3,16,4.00,3.5,435728.0,...,108932.00,205312.0,51328.00,55777.5,37.21,51.53,38.11,-78.57,-66.67,-81.82
7,20251001,15,Wednesday,3,3,3,10,3.33,3.0,243739.0,...,81246.33,74566.0,24855.33,20785.0,27.12,30.81,31.10,-78.57,-66.67,-86.36
8,20251001,16,Wednesday,2,2,2,2,1.00,1.0,33335.0,...,16667.50,8531.0,4265.50,4265.5,25.33,28.35,28.35,-85.71,-77.78,-90.91
9,20251001,17,Wednesday,2,2,2,2,1.00,1.0,37757.0,...,18878.50,12149.0,6074.50,6074.5,28.32,32.73,32.73,-85.71,-77.78,-90.91


In [9]:
# View summary statistics for key metrics
print("Revenue and Transaction Summary by Hour:")
attrs[['price_total_sum', 'trans_id_count', 'transaction_mean']].describe()

Revenue and Transaction Summary by Hour:


,price_total_sum,trans_id_count,transaction_mean
count,3.010000e+02,301.000000,301.000000
mean,1.777270e+05,2.023256,84974.011661
std,1.831490e+05,1.279111,67527.152871
min,1.801000e+03,1.000000,1801.000000
25%,5.267600e+04,1.000000,38520.000000
50%,1.205860e+05,2.000000,67531.000000
75%,2.359650e+05,3.000000,110968.000000
max,1.015430e+06,7.000000,496607.000000


In [10]:
# View cross-model comparison metrics (hour vs day)
print("Hour vs Day Comparison Metrics:")
comparison_cols = [
    'customer_id_vs_daily_count_pct', 
    'product_id_vs_daily_count_pct', 
    'trans_id_vs_daily_count_pct'
]
attrs[comparison_cols].describe()

Hour vs Day Comparison Metrics:


,customer_id_vs_daily_count_pct,product_id_vs_daily_count_pct,trans_id_vs_daily_count_pct
count,301.000000,301.000000,301.000000
mean,-82.516777,-79.415615,-90.033156
std,9.901556,11.412598,5.990918
min,-92.860000,-90.000000,-97.140000
25%,-90.910000,-88.890000,-94.440000
50%,-84.620000,-80.000000,-92.310000
75%,-76.920000,-75.000000,-87.500000
max,-44.440000,-33.330000,-66.670000


## 8. Export to Excel

In [11]:
# Export model results to Excel
exporter = ExcelExporter(ctx)
output_file = f'outputs/{cfg_model["model_name"]}_export.xlsx'
exporter.export_model(cfg_model['model_name'], output_file, include_input=True)

print(f"✓ Export complete: {output_file}")
print("\nExcel tabs:")
print(f"  1. {cfg_model['input_dataset_name']} (input)")
print(f"  2. {cfg_model['model_name']}_filters")
print(f"  3. {cfg_model['model_name']}_attrs")

✓ Export complete: outputs/daily_hour_export.xlsx

Excel tabs:
  1. transactions_filters (input)
  2. daily_hour_filters
  3. daily_hour_attrs


## 9. Save Context State

Save the complete context state for use in downstream notebooks:

In [12]:
# Save context state (datasets, config, metadata)
state_dir = save_context_state(ctx=ctx, base_cfg=base_cfg)

print(f"✓ Context state saved: {state_dir}")
print(f"  - Total datasets: {len(ctx.datasets)}")
print(f"\nTo load this state in another notebook:")
print(f"  from src.core.persistence import load_context_state")
print(f"  ctx, base_cfg = load_context_state('{state_dir}')")

✓ Context state saved: data\context_states\test_client_20251022_150907
  - Total datasets: 7

To load this state in another notebook:
  from src.core.persistence import load_context_state
  ctx, base_cfg = load_context_state('data\context_states\test_client_20251022_150907')
